In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Set paths
dataset_path = "dataset/train/"
augmented_path = "dataset/augmented_train/"

In [3]:

# Define categories
categories = ["healthy", "powdery_mildew", "new_disease"]

In [4]:
# Create output folders
for category in categories:
    os.makedirs(os.path.join(augmented_path, category), exist_ok=True)

In [5]:
# Data augmentation settings
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5],
    fill_mode="nearest"
)

In [6]:
# Generate images
for category in categories:
    img_folder = os.path.join(dataset_path, category)
    save_folder = os.path.join(augmented_path, category)

    for img_name in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize images to 224x224
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=save_folder, save_prefix="aug", save_format="jpg"):
            i += 1
            if i >= 50:  # Generate 50 new images per original image
                break

print("Data Augmentation Complete")

Data Augmentation Complete


In [7]:
import os
import shutil
import random

# Paths
augmented_dataset = "dataset/augmented_train/"
output_dataset = "dataset_final/"

# Create train/val/test folders
for subset in ["train", "val", "test"]:
    for category in categories:
        os.makedirs(os.path.join(output_dataset, subset, category), exist_ok=True)

# Split dataset
for category in categories:
    img_files = os.listdir(os.path.join(augmented_dataset, category))
    random.shuffle(img_files)

    train_split = int(0.8 * len(img_files))
    val_split = int(0.9 * len(img_files))

    for i, file in enumerate(img_files):
        src_path = os.path.join(augmented_dataset, category, file)

        if i < train_split:
            dest_path = os.path.join(output_dataset, "train", category, file)
        elif i < val_split:
            dest_path = os.path.join(output_dataset, "val", category, file)
        else:
            dest_path = os.path.join(output_dataset, "test", category, file)

        shutil.copy(src_path, dest_path)

print("Dataset Split Completed")


Dataset Split Completed


In [8]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
yaml_content = """train: C:/Users/supereme/Documents/fyp_rbd/dataset_final/train
val: C:/Users/supereme/Documents/fyp_rbd/dataset_final/val
test: C:/Users/supereme/Documents/fyp_rbd/dataset_final/test  # Optional
nc: 3
names: ["healthy", "powdery_mildew", "new_disease"]

"""

with open("dataset_final/data.yaml", "w") as file:
    file.write(yaml_content)

print("data.yaml file created successfully!")



data.yaml file created successfully!


In [10]:
from ultralytics import YOLO

# Load YOLOv8 classification model
model = YOLO("yolov8n-cls.pt")  # Load the smallest YOLOv8 classification model
model.train(data="C:/Users/supereme/Documents/fyp rbd/dataset_final/", epochs=50, imgsz=224, batch=16)


New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.75  Python-3.12.2 torch-2.4.1+cpu CPU (Intel Core(TM) i3-8130U 2.20GHz)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=C:/Users/supereme/Documents/fyp rbd/dataset_final/, epochs=50, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train23, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False

train: Scanning C:\Users\supereme\Documents\fyp rbd\dataset_final\train... 3378 images, 0 corrupt: 100%|██████████| 337


train: New cache created: C:\Users\supereme\Documents\fyp rbd\dataset_final\train.cache


val: Scanning C:\Users\supereme\Documents\fyp rbd\dataset_final\val... 945 images, 0 corrupt: 100%|██████████| 945/945 


val: New cache created: C:\Users\supereme\Documents\fyp rbd\dataset_final\val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs\classify\train23
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

                   all          1          1



50 epochs completed in 2.750 hours.
Optimizer stripped from runs\classify\train23\weights\last.pt, 3.0MB
Optimizer stripped from runs\classify\train23\weights\best.pt, 3.0MB

Validating runs\classify\train23\weights\best.pt...
Ultralytics 8.3.75  Python-3.12.2 torch-2.4.1+cpu CPU (Intel Core(TM) i3-8130U 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: C:\Users\supereme\Documents\fyp rbd\dataset_final\train... found 3378 images in 3 classes  
val: C:\Users\supereme\Documents\fyp rbd\dataset_final\val... found 945 images in 3 classes  
test: C:\Users\supereme\Documents\fyp rbd\dataset_final\test... found 939 images in 3 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 30/30 [00:11<00:00,  2.62it/s]


                   all          1          1
Speed: 0.0ms preprocess, 9.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train23


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000220A2BEA5D0>
curves: []
curves_results: []
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
save_dir: WindowsPath('runs/classify/train23')
speed: {'preprocess': 0.0006219048161434118, 'inference': 9.166588042289876, 'loss': 5.100537140769933e-05, 'postprocess': 0.00013989421339439495}
task: 'classify'
top1: 1.0
top5: 1.0

In [11]:
pip install onnx onnxruntime onnxslim


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from ultralytics import YOLO

model = YOLO("runs/classify/train21/weights/best.pt")  # Make sure the path is correct
print("Model loaded successfully!")

Model loaded successfully!


In [13]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:

#Evaluate the model on the test set
metrics = model.val(data="dataset_final")


Ultralytics 8.3.75  Python-3.12.2 torch-2.4.1+cpu CPU (Intel Core(TM) i3-8130U 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: C:\Users\supereme\Documents\fyp rbd\dataset_final\train... found 3378 images in 3 classes  
val: C:\Users\supereme\Documents\fyp rbd\dataset_final\val... found 945 images in 3 classes  
test: C:\Users\supereme\Documents\fyp rbd\dataset_final\test... found 939 images in 3 classes  


val: Scanning C:\Users\supereme\Documents\fyp rbd\dataset_final\val... 945 images, 0 corrupt: 100%|██████████| 945/945 
               classes   top1_acc   top5_acc: 100%|██████████| 60/60 [00:10<00:00,  5.78it/s]


                   all          1          1
Speed: 0.0ms preprocess, 7.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val


In [15]:
from ultralytics import YOLO
from sklearn.metrics import classification_report, accuracy_score
import torch
import os
import cv2
import numpy as np
import pandas as pd  # Import pandas for table format

# Load trained YOLOv8 classification model
model = YOLO("runs/classify/train21/weights/best.pt")  # Update path if needed

# Define test dataset path
test_path = "dataset_final/test"

# Get class names as a list
class_names = list(model.names.values())  # Convert dict to list

# Initialize lists for true and predicted labels
true_labels = []
pred_labels = []

# Loop through test dataset
for category in os.listdir(test_path):
    category_path = os.path.join(test_path, category)

    for img_name in os.listdir(category_path):
        if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue  # Skip non-image files

        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f" Error: Could not read {img_path}, skipping.")
            continue

        # Preprocess the image
        img = cv2.resize(img, (224, 224))
        img = np.transpose(img, (2, 0, 1))  # Change to (C, H, W)
        img = torch.tensor(img, dtype=torch.float32) / 255.0  # Normalize
        img = img.unsqueeze(0)  # Add batch dimension

        # Get model prediction
        results = model(img)
        predicted_class = results[0].probs.top1  # Get top prediction index

        # Convert category name to index using the updated list
        if category in class_names:
            true_labels.append(class_names.index(category))
            pred_labels.append(predicted_class)
        else:
            print(f"Warning: Class '{category}' not found in model names.")

# Convert to NumPy arrays
true_labels = np.array(true_labels)
pred_labels = np.array(pred_labels)

# Calculate Precision, Recall, F1-Score, and Accuracy
report_dict = classification_report(true_labels, pred_labels, target_names=class_names, digits=4, output_dict=True)
accuracy = accuracy_score(true_labels, pred_labels)

# Convert metrics to DataFrame for table format
df_report = pd.DataFrame(report_dict).transpose()





0: 224x224 healthy 1.00, new_disease 0.00, powdery_mildew 0.00, 31.5ms
Speed: 0.0ms preprocess, 31.5ms inference, 2.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 0.99, new_disease 0.01, powdery_mildew 0.00, 25.2ms
Speed: 0.0ms preprocess, 25.2ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 1.00, new_disease 0.00, powdery_mildew 0.00, 25.2ms
Speed: 0.0ms preprocess, 25.2ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 1.00, new_disease 0.00, powdery_mildew 0.00, 35.1ms
Speed: 0.0ms preprocess, 35.1ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 0.99, new_disease 0.01, powdery_mildew 0.00, 33.8ms
Speed: 0.0ms preprocess, 33.8ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 0.97, new_disease 0.03, powdery_mildew 0.00, 74.6ms
Speed: 0.1ms preprocess, 74.6ms inference, 0.6ms postprocess per image at shape (1,

In [16]:
#  Print table
print("Classification Report:")
print(df_report)

# Print accuracy separately
print(" Accuracy:", accuracy)

Classification Report:
                precision    recall  f1-score     support
healthy          0.873900  0.970684  0.919753  307.000000
new_disease      0.875346  1.000000  0.933530  316.000000
powdery_mildew   1.000000  0.750000  0.857143  316.000000
accuracy         0.906283  0.906283  0.906283    0.906283
macro avg        0.916416  0.906895  0.903475  939.000000
weighted avg     0.916823  0.906283  0.903319  939.000000
 Accuracy: 0.906283280085197


In [17]:
from ultralytics import YOLO

model = YOLO("runs/classify/train21/weights/best.pt")  
results = model("C:/Users/supereme/Pictures/Screenshots/t1.png") 
# Print classification results
print(results[0].probs.top1)  # Predicted class index
print(results[0].names[results[0].probs.top1])  # Predicted class name


image 1/1 C:\Users\supereme\Pictures\Screenshots\t1.png: 224x224 new_disease 0.83, healthy 0.12, powdery_mildew 0.05, 12.8ms
Speed: 35.4ms preprocess, 12.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
1
new_disease


In [18]:
import time
import torch

# ---------- 1. Model Size (MB) ----------
model_path = "runs/classify/train21/weights/best.pt"
model_size_MB = os.path.getsize(model_path) / (1024 * 1024)  # Convert bytes to MB
print(f"Model Size: {model_size_MB:.2f} MB")

# ---------- 2. Number of Parameters (M) ----------
# Load the PyTorch model directly
yolo_model = torch.load(model_path, map_location="cpu")  # Load to CPU to avoid GPU dependency
if isinstance(yolo_model, dict) and 'model' in yolo_model:
    model_params = sum(p.numel() for p in yolo_model['model'].parameters())
else:
    model_params = sum(p.numel() for p in yolo_model.parameters())
print(f"Number of Parameters: {model_params / 1e6:.2f} Million")

# ---------- 3. Training Time per Epoch (rough estimation from logs) ----------
# You can parse the training logs for precise timing or use a rough estimation here
train_log = "runs/classify/train21/results.csv"
if os.path.exists(train_log):
    import pandas as pd
    df = pd.read_csv(train_log)
    if "epoch" in df.columns and "time/epoch" in df.columns:
        avg_epoch_time = df["time/epoch"].mean()
        print(f"Average Training Time per Epoch: {avg_epoch_time:.2f} seconds")
    else:
        print("Warning: 'time/epoch' column not found in training log.")
else:
    print("Training log not found to estimate training time.")

# ---------- 4. Testing Run Time (Total and Average per image) ----------
import glob

test_images = glob.glob(test_path + "/**/*.jpg", recursive=True) + \
              glob.glob(test_path + "/**/*.jpeg", recursive=True) + \
              glob.glob(test_path + "/**/*.png", recursive=True)

start_time = time.time()
for img_path in test_images:
    img = cv2.imread(img_path)
    if img is None:
        continue
    img = cv2.resize(img, (224, 224))
    img = np.transpose(img, (2, 0, 1))
    img = torch.tensor(img, dtype=torch.float32) / 255.0
    img = img.unsqueeze(0)
    _ = model(img)  # Inference
end_time = time.time()

total_inference_time = end_time - start_time
avg_inference_time = total_inference_time / len(test_images)

print(f"Total Test Runtime: {total_inference_time:.2f} seconds")
print(f"Average Inference Time per Image: {avg_inference_time:.4f} seconds")


Model Size: 2.83 MB
Number of Parameters: 1.44 Million

0: 224x224 healthy 1.00, new_disease 0.00, powdery_mildew 0.00, 21.0ms
Speed: 0.0ms preprocess, 21.0ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 0.99, new_disease 0.01, powdery_mildew 0.00, 22.5ms
Speed: 0.3ms preprocess, 22.5ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 1.00, new_disease 0.00, powdery_mildew 0.00, 21.7ms
Speed: 0.0ms preprocess, 21.7ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 1.00, new_disease 0.00, powdery_mildew 0.00, 24.0ms
Speed: 0.0ms preprocess, 24.0ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 0.99, new_disease 0.01, powdery_mildew 0.00, 21.5ms
Speed: 0.0ms preprocess, 21.5ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 healthy 0.97, new_disease 0.03, powdery_mildew 0.00, 27.3ms
Speed: 0.0ms preprocess, 27.

In [19]:
summary_df = pd.DataFrame({
    "Metric": [
        "Model Size (MB)",
        "Parameters (M)",
        "Avg Train Time/Epoch (s)",
        "Total Test Runtime (s)",
        "Avg Inference Time (s)"
    ],
    "Value": [
        f"{model_size_MB:.2f}",
        f"{model_params / 1e6:.2f}",
        f"{avg_epoch_time:.2f}" if 'avg_epoch_time' in locals() else "N/A",
        f"{total_inference_time:.2f}",
        f"{avg_inference_time:.4f}"
    ]
})

print("\n--- Model Summary ---")
print(summary_df)



--- Model Summary ---
                     Metric   Value
0           Model Size (MB)    2.83
1            Parameters (M)    1.44
2  Avg Train Time/Epoch (s)     N/A
3    Total Test Runtime (s)   31.32
4    Avg Inference Time (s)  0.0334
